In [1]:
#pip install pytorch_pretrained_bert pytorch-nlp
#pip install transformers


In [2]:
#conda install -c conda-forge gensim
#pip install --upgrade gensim   

In [3]:
text = "After stealing money from the bank vault, the bank robber was seen fishing on the Mississippi river bank."

In [4]:
import gensim
from gensim.models.word2vec import Word2Vec

In [5]:
sentences = text
w2v_model = Word2Vec(sentences)

In [6]:
words = list(w2v_model.wv.index_to_key)
print(words)

[' ', 'e', 'n', 'i', 's', 'a', 'r', 't', 'b']


In [7]:
w2v_model.save('model.bin')


In [8]:
new_model = Word2Vec.load('model.bin')
print(new_model)

Word2Vec<vocab=9, vector_size=100, alpha=0.025>


In [9]:
import sys
import numpy as np
import random as rn
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
from pytorch_pretrained_bert import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output
import pandas as pd
import matplotlib.pyplot as plt


In [10]:


#from transformers import BertTokenizer
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [11]:
from transformers import __version__; print(__version__)

4.29.0


In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [13]:
tokenize_ = tokenizer.tokenize(text)
print("Text after tokenization: ")
print(tokenize_)
max_len = 25

textLst = tokenize_[:max_len-2]
input_sequence = ["[CLS]"] + textLst + ["[SEP]"]
pad_len = max_len - len(input_sequence)

print("After adding [CLS] and [SEP]: ")
print(input_sequence)
tokens = tokenizer.convert_tokens_to_ids(input_sequence)
print("After converting Tokens to Id: ")
print(tokens)
tokens += [0] * pad_len
print("tokens: ")
print(tokens)
pad_masks = [1] * len(input_sequence) + [0] * pad_len
print("Pad Masking: ")
print(pad_masks)
segment_ids = [0] * max_len
print("Segment Ids: ")
print(segment_ids)

Text after tokenization: 
['after', 'stealing', 'money', 'from', 'the', 'bank', 'vault', ',', 'the', 'bank', 'robber', 'was', 'seen', 'fishing', 'on', 'the', 'mississippi', 'river', 'bank', '.']
After adding [CLS] and [SEP]: 
['[CLS]', 'after', 'stealing', 'money', 'from', 'the', 'bank', 'vault', ',', 'the', 'bank', 'robber', 'was', 'seen', 'fishing', 'on', 'the', 'mississippi', 'river', 'bank', '.', '[SEP]']
After converting Tokens to Id: 
[101, 2044, 11065, 2769, 2013, 1996, 2924, 11632, 1010, 1996, 2924, 27307, 2001, 2464, 5645, 2006, 1996, 5900, 2314, 2924, 1012, 102]
tokens: 
[101, 2044, 11065, 2769, 2013, 1996, 2924, 11632, 1010, 1996, 2924, 27307, 2001, 2464, 5645, 2006, 1996, 5900, 2314, 2924, 1012, 102, 0, 0, 0]
Pad Masking: 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
Segment Ids: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)
segments_ids = [1] * len(tokenized_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Print out the tokens.
print (tokenized_text)

['[CLS]', 'after', 'stealing', 'money', 'from', 'the', 'bank', 'vault', ',', 'the', 'bank', 'robber', 'was', 'seen', 'fishing', 'on', 'the', 'mississippi', 'river', 'bank', '.', '[SEP]']


In [15]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|███████████████████████████████████████████████████████████████████████████████| 662804195/662804195 [08:12<00:00, 1346438.48B/s]


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Li

In [16]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(encoded_layers, dim=0)

# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([22, 12, 768])

In [ ]:
# Stores the token vectors, with shape [23 x 768]
token_vecs_sum = []

# `token_embeddings` is a [23 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

In [22]:
# Stores the token vectors, with shape [23 x 768]
token_vecs_sum = []

# `token_embeddings` is a [23 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 22 x 768


In [23]:
# `encoded_layers` has shape [12 x 1 x 23 x 768]

# `token_vecs` is a tensor with shape [23 x 768]
token_vecs = encoded_layers[11][0]

# Calculate the average of all 23 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
     

In [24]:
print ("Our final sentence embedding vector of shape:", sentence_embedding.size())


Our final sentence embedding vector of shape: torch.Size([768])


In [25]:
sentence_embedding

tensor([ 2.1358e-01, -6.3680e-01,  6.5889e-01,  1.8925e-02, -3.9966e-01,
        -3.7560e-01, -4.7043e-02, -9.8560e-02,  1.5726e-01,  5.8103e-01,
        -2.9570e-01,  1.9844e-01,  3.5090e-01,  2.4400e-01,  1.6944e-01,
        -3.4082e-01,  2.2248e-01,  1.9042e-01, -7.6469e-02,  5.7022e-01,
        -2.9180e-01,  7.1141e-02,  2.5399e-01,  5.2583e-01,  1.1742e-01,
         5.8425e-02, -1.4689e-01,  3.7150e-01,  7.4130e-02, -9.4014e-01,
         3.2702e-01,  8.9699e-02,  5.6249e-02,  4.0415e-01, -1.9911e-02,
         1.4569e-01, -3.2897e-01,  1.0943e-01, -5.1025e-02, -7.1243e-01,
         1.2372e-01, -6.4714e-02, -1.0946e-01,  2.0264e-01, -1.5510e-01,
        -8.0698e-01,  3.9742e-01,  1.5526e-01, -3.4541e-01,  2.1713e-01,
        -3.1845e-02, -3.1141e-01,  3.5776e-01, -2.0226e-01, -4.1421e-01,
        -1.7628e-05,  2.1077e-01,  1.1151e-01,  2.2985e-02,  2.5953e-02,
         1.0885e-01,  3.1563e-01, -1.3581e-02,  1.4987e-01, -3.4291e-01,
         6.5344e-01, -8.8557e-01,  9.2735e-02,  9.0